In [1]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('t5-small')

print(tokenizer)

#编码试算
print(
    tokenizer.batch_encode_plus(
        ['Hello, this one sentence!', 'This is another sentence.']))

#label的编码方式,但是试验结果是和input的编码方式一样,没有区别
with tokenizer.as_target_tokenizer():
    print(
        tokenizer.batch_encode_plus(
            ['Hello, this one sentence!', 'This is another sentence.']))

PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<ex

In [2]:
from datasets import load_dataset, load_from_disk

#加载数据集
#dataset = load_dataset('xsum')
dataset = load_from_disk('datas/xsum')

#采样,数据量太大了跑不动
dataset['train'] = dataset['train'].shuffle(1).select(range(20000))
dataset['validation'] = dataset['validation'].shuffle(1).select(range(100))
dataset['test'] = dataset['test'].shuffle(1).select(range(100))

print(dataset['train'][0])

dataset

Loading cached shuffled indices for dataset at datas/xsum/train/cache-5a721ded3b377201.arrow
Loading cached shuffled indices for dataset at datas/xsum/validation/cache-a35139f4dcb69b4b.arrow
Loading cached shuffled indices for dataset at datas/xsum/test/cache-522a74c9bc704175.arrow


{'document': "Clay, who has agreed a two-year deal, made 39 appearances for Scottish Premiership club Motherwell last season after joining them in June 2016.\nThe 25-year-old had spent the two previous seasons with Grimsby, playing 74 National League games.\nClay is Leyton Orient's ninth signing since being relegated from League Two last season.", 'summary': 'National League side Leyton Orient have signed Motherwell midfielder Craig Clay on a free transfer.', 'id': '40635923'}


DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 100
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 100
    })
})

In [3]:
#数据预处理函数,分词
def f(examples):
    #编码input
    data = tokenizer.batch_encode_plus(
        #在输入的前面加summarize前缀,这个是h5模型识别任务类型的标识
        ['summarize: ' + i for i in examples['document']],
        max_length=1024,
        truncation=True,
    )

    #编码label
    with tokenizer.as_target_tokenizer():
        data['labels'] = tokenizer.batch_encode_plus(
            examples['summary'],
            max_length=128,
            truncation=True,
        )['input_ids']

    return data


dataset = dataset.map(function=f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['document', 'summary', 'id'])

print(dataset['train'][0])

dataset

Loading cached processed dataset at datas/xsum/train/cache-495b96586a2de104.arrow


Loading cached processed dataset at datas/xsum/train/cache-b84649ce42fe08dd.arrow


Loading cached processed dataset at datas/xsum/train/cache-2f914e8128c67ce2.arrow


Loading cached processed dataset at datas/xsum/train/cache-674ddaa89e123e9b.arrow


Loading cached processed dataset at datas/xsum/validation/cache-efcb6687daf3d8ba.arrow


...

Loading cached processed dataset at datas/xsum/test/cache-4c8896e0ad2e5251.arrow


Loading cached processed dataset at datas/xsum/test/cache-b2007e056428bf6c.arrow


Loading cached processed dataset at datas/xsum/test/cache-16b2d4bdf57a8a24.arrow


{'input_ids': [21603, 10, 20988, 6, 113, 65, 4686, 3, 9, 192, 18, 1201, 1154, 6, 263, 6352, 3179, 7, 21, 12580, 6552, 2009, 1886, 8007, 2091, 336, 774, 227, 6109, 135, 16, 1515, 4619, 37, 944, 18, 1201, 18, 1490, 141, 1869, 8, 192, 1767, 9385, 28, 23427, 7, 969, 6, 1556, 3, 4581, 868, 3815, 1031, 5, 20988, 19, 312, 21220, 3, 16495, 31, 7, 24651, 8097, 437, 271, 3, 60, 8791, 26, 45, 3815, 2759, 336, 774, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [868, 3815, 596, 312, 21220, 3, 16495, 43, 3814, 8007, 2091, 2076, 1846, 49, 12870, 20988, 30, 3, 9, 339, 2025, 5, 1]}


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [4]:
#这个函数和下面这个工具类等价,但我也是模仿实现的,不确定有没有出入
#from transformers import DataCollatorForSeq2Seq
#DataCollatorForSeq2Seq(tokenizer, model=model)

import torch


#数据批处理函数
def collate_fn(data):
    #求最长的label
    max_length = max([len(i['labels']) for i in data])

    #把所有的label都补pad到最长
    for i in data:
        pads = [-100] * (max_length - len(i['labels']))
        i['labels'] = i['labels'] + pads

    #把多个数据整合成一个tensor
    data = tokenizer.pad(
        encoded_inputs=data,
        padding=True,
        max_length=None,
        pad_to_multiple_of=None,
        return_tensors='pt',
    )

    #定义decoder_input_ids
    data['decoder_input_ids'] = torch.zeros_like(data['labels'],
                                                 dtype=torch.long)
    data['decoder_input_ids'][:, 1:] = data['labels'][:, :-1]
    data['decoder_input_ids'][data['decoder_input_ids'] == -100] = 0

    return data


data = [{
    'input_ids': [21603, 10, 37, 3719, 13],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [10455, 120, 80]
}, {
    'input_ids': [21603, 10, 7086, 8408, 563],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [301, 53, 4074, 1669]
}]

collate_fn(data)

{'input_ids': tensor([[21603,    10,    37,  3719,    13],
        [21603,    10,  7086,  8408,   563]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]), 'labels': tensor([[10455,   120,    80,  -100],
        [  301,    53,  4074,  1669]]), 'decoder_input_ids': tensor([[    0, 10455,   120,    80],
        [    0,   301,    53,  4074]])}

In [5]:
#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([8, 1024])
attention_mask torch.Size([8, 1024])
labels torch.Size([8, 35])
decoder_input_ids torch.Size([8, 35])


2500

In [6]:
from transformers import AutoModelForSeq2SeqLM, T5Model

#加载模型
#model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')


#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = T5Model.from_pretrained('t5-small')

        #本来应该写tokenizer.vocab_size=32100就可以了。
        #但是预训练模型里的参数是32128，所以为了方便就直接使用这个尺寸了
        self.fc = torch.nn.Linear(512, 32128, bias=False)

        #加载预训练模型的参数
        parameters = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
        self.fc.load_state_dict(parameters.lm_head.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

    def forward(self, input_ids, attention_mask, labels, decoder_input_ids):

        logits = self.pretrained.encoder(input_ids=input_ids,
                                         attention_mask=attention_mask)

        logits = logits.last_hidden_state

        logits = self.pretrained.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=logits,
            encoder_attention_mask=attention_mask,
        )

        logits = logits.last_hidden_state

        logits = logits * (512**-0.5)

        logits = self.fc(logits)

        loss = self.criterion(logits.reshape(-1, 32128),
                              labels.reshape(-1))

        return {'loss': loss, 'logits': logits}


model = Model()

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

#模型试算
out = model(**data)

out['loss'], out['logits'].shape

7695.616


(tensor(3.9883, grad_fn=<NllLossBackward0>), torch.Size([8, 35, 32128]))

In [7]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=4,
        collate_fn=collate_fn,
        shuffle=True,
        drop_last=True,
    )

    for i, data in enumerate(loader_test):
        break

    #计算
    with torch.no_grad():
        out = model(**data)

    for i in range(4):
        input_ids = tokenizer.decode(data['input_ids'][i])
        pred = tokenizer.decode(out['logits'].argmax(dim=2)[i]),
        label = tokenizer.decode(data['decoder_input_ids'][i])

        #print('input_ids=', input_ids)
        print('pred=', pred)
        print('label=', label)
        print()


test()

pred= ('the memorial will be placed near the of thea man who fell in  into the river.  the  the the the the the the the the the the the the the the',)
label= <pad> A monument will be built in memory of a student who died after falling into the River Avon.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

pred= (' trialsting child citizensex trafficender  in to have for agentscover agents  sex  a boy    picturesnography images  the   ',)
label= <pad> A convicted British sex offender caught trying to pay US undercover officers for sex with a boy has admitted taking pornographic images into the country.

pred= (' say investigating to arrest the73 49 named the death of a 21er  t   : the the the the the the the the the the the the the the',)
label= <pad> Police are continuing to question a man over the murder of a pensioner in Bridgend county.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

pred= ('charity figures of patients waiting to tr

In [8]:
from transformers.trainer_pt_utils import get_parameter_names
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    #一个工具函数,能够获取模型中所有参数的名字列表
    parameter_names = get_parameter_names(model, [torch.nn.LayerNorm])

    #定义哪些参数参与weight_decay
    parameter_names = [i for i in parameter_names if 'bias' not in i]

    #根据参数要参与weight_decay来把参数分为两组
    parameter_names = [
        {
            'params':
            [p for i, p in model.named_parameters() if i in parameter_names],
            'weight_decay':
            1e-2,
        },
        {
            'params': [
                p for i, p in model.named_parameters()
                if i not in parameter_names
            ],
            'weight_decay':
            0.0,
        },
    ]

    #定义优化器
    optimizer = AdamW(parameter_names, betas=(0.9, 0.999), eps=1e-8, lr=2e-5)

    #定义lr调整器
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    model.train()
    for i, data in enumerate(loader):
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            pred = tokenizer.decode(out['logits'].argmax(dim=2)[0]),

            label = tokenizer.decode(data['decoder_input_ids'][0])

            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(i, loss.item(), lr)
            print('pred=', pred)
            print('label=', label)
            print()

        torch.save(model, 'models/4.长求总.model')


train()

/root/anaconda3/envs/cpu/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 3.93034291267395 1.9992e-05
pred= ('the one  is the  is  ed as year  part of the breeding  breedingover the  the the the the the the the the the the the the the the the the',)
label= <pad> Every single creature at Chester Zoo is being counted this week as part of its annual stock take.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

50 3.4090747833251953 1.9592e-05
pred= ('whale was the first moment whenspotted whalehumpback whale breachingd off off the sea  off from thea whale  of  </s>  whale whale whale whale whale whale whale whale whale whale whale',)
label= <pad> This is the spectacular moment a humpback whale breached high into the air just metres from a boat full of tourists.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

100 2.6107447147369385 1.9192000000000002e-05
pred= ('Bennett judgeminhian judge, wasallegedlyaped a girlyear-old girl -d childs child, hisa suspected was been jailed for life yearsand-a-half years.</s>',)
la

1050 2.843055009841919 1.1592000000000002e-05
pred= ('Bristol West has  the own to  Bristol railtransie transit buses scheme route to the.</s></s> Bristol The The The The The The The The The The The  The The The The The The The The The The The A The The The The',)
label= <pad> The government has given its approval to the planned "rapid transit" bus route through Bristol.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

1100 3.3547728061676025 1.1192e-05
pred= ('The- Ireland- associations have secured a £ fund50m- Choice  Union Fund of</s></s> the The The The The The The The The The The The The A',)
label= <pad> Two Northern Ireland housing bodies have secured a combined £280m from the European Investment Bank.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

1150 3.3273966312408447 1.0792000000000001e-05
pred= ('Greece PrimeChancellor Angela Merkel has said she

2000 2.6884877681732178 3.992e-06
pred= ('The council committee received  ised by failing it has education with extra needs needs in</s>: The The The The The The The The The A The The The The The The The The The',)
label= <pad> A council has been repeatedly criticised for how it handles children with special educational needs.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

2050 3.6314761638641357 3.5920000000000005e-06
pred= ('A  gngynoedd onddy ytholiad  yngw   e ahi an ai y ngi  yth yiw yynd r  ywdd  d   r y yaa yynos  o  angeh yi  yn bby<unk>ai</s></s></s></s></s></s></s></s></s></s></s></s></s> </s>',)
label= <pad> Ers y cyhoeddiad bod etholiad ar y gweill, mae rhai yn dadlau bod Cymru ar ei cholled, gyda'r wasg Brydeinig weithiau'n anghofio cyfeirio at y ffaith nad yw pob polisi neu addewid yn berthnasol yr

2100 2.844595193862915 3.192e-06
pred= ('The sea scientistfoodl"" is being held by a " death of  on   of thea er\'.</s></s

In [9]:
model = torch.load('models/4.长求总.model')
test()

pred= ('Swansea City hass plans has pleased that will be made in the with  the stadium Stadium.</s></s> The The The The The The Swan Swan Swan Swan Swan Swan',)
label= <pad> Swansea council's leader is optimistic progress can be made during talks about expanding the Liberty Stadium.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

pred= ('A President candidate candidate Pault Romney is looking to be  own presidentpresidential candidate mate,.-    of</s></s>   Republican Republican Republican Republican',)
label= <pad> US Republican presidential candidate Mitt Romney is expected to name his vice-presidential running mate soon - possibly within days.</s><pad><pad><pad><pad><pad><pad>

pred= ('A woman doctor haswhose car was found in  sea in  s andrelated injuries" has family has said.</s></s> Tor A A A A A A A A A',)
label= <pad> A junior doctor whose body was found in the sea suffered from "work-related anxiety", her sister has said.</s><pad><pad><pad><pad><pad><pad><pad